<a href="https://colab.research.google.com/github/dakilaledesma/NCBG/blob/main/tools/Testset_Image_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
! cp /content/drive/MyDrive/UNC/NCBG/qry_fsus_fulllistimg_ai.xlsx /content/qry_fsus_fulllistimg_ai.xlsx
! cp /content/drive/MyDrive/UNC/NCBG/qry_fsus_trainingdata_ai.xlsx /content/qry_fsus_trainingdata_ai.xlsx

In [2]:
! pip install joblib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
import pandas as pd

train_df = pd.read_excel("qry_fsus_trainingdata_ai.xlsx")
full_df = pd.read_excel("qry_fsus_fulllistimg_ai.xlsx")

In [32]:
full_df = full_df[["filename", "directoryname"]]

In [33]:
print(full_df.head())

            filename directoryname
0  1413831366460.jpg           gmp
1  1413831418011.jpg           gmp
2  1413831438890.jpg           gmp
3  1413833265035.jpg           gmp
4  1413833627833.jpg           gmp


In [34]:
train_df = train_df[["file", "directory"]]

In [35]:
train_df = train_df.rename(columns={"file": "filename", "directory": "directoryname"})

In [36]:
print(train_df.head())

                 filename directoryname
0  justice_slide_0001.jpg           wsj
1  justice_slide_0002.jpg           wsj
2  justice_slide_0004.jpg           wsj
3  justice_slide_0006.jpg           wsj
4  justice_slide_0007.jpg           wsj


In [43]:
print(len(full_df), len(train_df))

103333 8613


In [72]:
test_df = pd.merge(full_df, train_df, "outer", indicator=True)
test_df = test_df[test_df["_merge"] == "left_only"][["filename", "directoryname"]]

In [74]:
print(len(test_df), test_df.head())

94800             filename directoryname
1  1413831418011.jpg           gmp
2  1413831438890.jpg           gmp
4  1413833627833.jpg           gmp
5  1413834182808.jpg           gmp
6  1413834376873.jpg           gmp


In [51]:
a = set()
b = set()

for idx, row in train_df.iterrows():
  a.add(f'{row["filename"]},{row["directoryname"]}')

for idx, row in full_df.iterrows():
  b.add(f'{row["filename"]},{row["directoryname"]}')

i = 0
for v in b:
  if v in a:
    i += 1

print(i)

8533


In [79]:
import os
from tqdm.notebook import tqdm
from urllib import request
import time
from collections import defaultdict
import io
from PIL import Image
from joblib import Parallel, delayed

if not os.path.isdir("images"):
      os.makedirs("images")

def down_ims(idx, row):
  im_bn = row["filename"]
  directory = row["directoryname"]
  im_url = f"https://fsus.ncbg.unc.edu/img/600/{directory}/{im_bn}"


  try:
    req = request.urlopen(request.Request(im_url), timeout=5).read()
  except:
    pass

  try:
    im = Image.open(io.BytesIO(req))
    im.thumbnail([600, 600])
    
    im.save(f"images/{idx}__{directory}_{im_bn}.jpg")
  except:
    pass

Parallel(n_jobs=4)(delayed(down_ims)(idx, row) for idx, row in tqdm(test_df.iterrows(), total=len(test_df)))

  0%|          | 0/94800 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [80]:
from glob import glob

print(len(list(glob("images/**/*.*", recursive=True))))

94670


In [81]:
! zip -r -q -0 /content/drive/MyDrive/UNC/NCBG/sept13_testset.zip /content/images